In [1]:
import numpy as np
import plotly.subplots as sp
import plotly.graph_objects as go

from model.model import City

In [2]:
# Set parameter values and run simulation
# Do param sweep with gamma (exponential), A and price (quadratic together)
num_steps  = 20
parameters = {
            'run_notes': 'Debugging model.',
            'subfolder': None,
            'width': 15,
            'height':15,

            # FLAGS
            'demographics_on': True,  # Set flag to False for debugging to check firm behaviour without demographics or housing market
            'center_city':     False, # Flag for city center in center if True, or bottom corner if False
            # 'random_init_age': False,  # Flag for randomizing initial age. If False, all workers begin at age 0
            'random_init_age': True,  # Flag for randomizing initial age. If False, all workers begin at age 0

            # LABOUR MARKET AND FIRM PARAMETERS
            'subsistence_wage': 40000., # psi
            'init_city_extent': 10.,    # CUT OR CHANGE?
            'seed_population': 400,
            'init_wage_premium_ratio': 0.2, # 1.2, ###

            # PARAMETERS MOST LIKELY TO AFFECT SCALE
            'c': 300.0,                            ###
            'price_of_output': 10,                 ######
            'density':600,                         #####
            'A': 3000,                             ### 
            'alpha': 0.18,
            'beta':  0.75,
            'gamma': 0.12, ### reduced from .14
            'overhead': 1,
            'mult': 1.2,
            'adjN': 0.15,
            'adjk': 0.05,
            'adjn': 0.25,
            'adjF': 0.15,
            'adjw': 0.15, 
            'dist': 1, 
            'init_agglomeration_population': 100000.0,
            'init_F': 100.0,
            'init_k': 100.0,
            'init_n': 100.0,

            # HOUSING AND MORTGAGE MARKET PARAMETERS
            'mortgage_period': 5.0,       # T, in years
            'working_periods': 40,        # in years
            'savings_rate': 0.3,
            'discount_rate': 0.07,        # 1/delta
            'r_prime': 0.05,
            'r_margin': 0.01,
            'property_tax_rate': 0.04,     # tau, annual rate, was c
            'housing_services_share': 0.3, # a
            'maintenance_share': 0.2,      # b
            'max_mortgage_share': 0.9,
            'ability_to_carry_mortgage': 0.28,
            'wealth_sensitivity': 0.1,
        }

def run_simulation(num_steps, parameters):
    city = City(num_steps, **parameters)
    city.run_model()

    agent_out = city.datacollector.get_agent_vars_dataframe()
    model_out = city.datacollector.get_model_vars_dataframe()
    return agent_out, model_out


agent_out, model_out = run_simulation(num_steps, parameters)
# Turn on for timing
# cProfile.run("agent_out, model_out = run_simulation(num_steps, parameters)", sort='cumulative')

In [3]:
# Setup variables for plotting

# Filter agent data
person_df = agent_out.query("agent_type == 'Person'").dropna(axis=1, how='all').reset_index(drop=True)
land_df = agent_out.query("agent_type == 'Land'").dropna(axis=1, how='all').reset_index(drop=True)

# Get time steps
# TODO get time steps from model df? Do I just need lenth/no time_steps
time_steps = person_df['time_step'].unique()
num_time_steps = len(time_steps)
time = np.arange(num_time_steps) # time array for the x-axis
person_middle_time_step = time_steps[num_time_steps // 2]


# Define the time steps and number of time steps to plot
time_steps_to_plot = 5

# # Get unique time steps from the person_df
# time_steps = person_df['time_step'].unique()
# num_time_steps = len(time_steps)

# Calculate the time step intervals
time_step_interval = max(1, num_time_steps // (time_steps_to_plot - 1))

In [4]:
# Heatmap grid fast, plotly - 5 plots 5 data

# Define the names for each row
row_names = ['Is Working', 'Warranted Price', 'Realized Price', 'Owner']

# Create subplots with Plotly
fig = sp.make_subplots(rows=4, cols=time_steps_to_plot, shared_yaxes=True)

# Define a custom color scale for the heatmap
color_scale = [[0, 'blue'], [1, 'red']]

# Create a list to store the titles for the columns
column_titles = []

# Set a fixed height for each row
row_height = 0.2  # You can adjust the height as needed

for i in range(time_steps_to_plot):
    time_step_index = min(i * time_step_interval, num_time_steps - 1)
    time_step = time_steps[time_step_index]
    person_df_at_time_step = person_df.query("time_step == @time_step")
    land_df_at_time_step = land_df.query("time_step == @time_step")

    # Calculate the number of rows and columns for each subplot
    num_rows, num_cols = 2, 3  # You can adjust these values to your preference

    # Create y-axis titles for the first plot in each row
    y_titles = [row_names[j] if j == 0 else '' for j in range(4)]

    # Create an Is Working Heatmap with square aspect ratio and a colorbar
    heatmap1 = go.Heatmap(
        x=person_df_at_time_step['x'],
        y=person_df_at_time_step['y'],
        z=person_df_at_time_step['is_working'],
        colorscale=color_scale,
        colorbar=dict(title='Is Working'),
        showlegend=(i == 0)  # Show legend for the first plot in each row
    )
    fig.add_trace(heatmap1, row=1, col=i+1)

    # Create a Warranted Price Heatmap with square aspect ratio and a colorbar
    heatmap2 = go.Heatmap(
        x=land_df_at_time_step['x'],
        y=land_df_at_time_step['y'],
        z=land_df_at_time_step['warranted_price'],
        colorscale='Viridis',
        colorbar=dict(title='Warranted Price'),
        showlegend=(i == 0)  # Show legend for the first plot in each row
    )
    fig.add_trace(heatmap2, row=2, col=i+1)

    # Create a Realized Price Heatmap with square aspect ratio and a colorbar
    heatmap3 = go.Heatmap(
        x=land_df_at_time_step['x'],
        y=land_df_at_time_step['y'],
        z=land_df_at_time_step['realized_price'],
        colorscale='Viridis',
        colorbar=dict(title='Realized Price'),
        showlegend=(i == 0)  # Show legend for the first plot in each row
    )
    fig.add_trace(heatmap3, row=3, col=i+1)

    # Add the time step as a column title
    column_titles.append(f'Time Step {time_step}')

# Update the layout to set y-axis titles
for j, title in enumerate(row_names):
    fig.update_yaxes(title_text=title, row=j+1, col=1)

# Update the layout to set column titles
fig.update_layout(
    title_text="Visualizations with Plotly",
    width=1000,
    height=1000,  # Adjust the height as needed
    title_x=0.5,  # Center the title
    title_y=0.97,  # Adjust the title position
)

for i, title in enumerate(column_titles):
    fig.update_xaxes(title_text=title, row=4, col=i+1)

fig.show()


In [5]:

def plot_model_data(model_out):
    workers = np.array(model_out['workers'])
    wage_premium = np.array(model_out['wage_premium'])
    wage = np.array(model_out['wage'])
    city_extent_calc = np.array(model_out['city_extent_calc'])
    p_dot = np.array(model_out['p_dot'])

    # Create subplots with Plotly
    fig = sp.make_subplots(rows=4, cols=2, subplot_titles=[
        'Evolution of the Wage (Rises)',
        'Evolution of the Workforce (Rises)',
        'Evolution of the City Extent (Rises)',
        'City Extent and Workforce (Curves Up)',
        'City Extent and Wage (Curves Up)',
        'Workforce Response to Wage',
        'Evolution of P-Dot'
    ])

    # Add traces for each subplot
    fig.add_trace(go.Scatter(x=time, y=wage, mode='lines'), row=1, col=1)
    fig.add_trace(go.Scatter(x=time, y=workers, mode='lines'), row=1, col=2)
    fig.add_trace(go.Scatter(x=time, y=city_extent_calc, mode='lines'), row=2, col=1)
    fig.add_trace(go.Scatter(x=city_extent_calc, y=workers, mode='lines'), row=2, col=2)
    fig.add_trace(go.Scatter(x=time, y=city_extent_calc, mode='lines'), row=3, col=1)
    fig.add_trace(go.Scatter(x=wage, y=workers, mode='lines'), row=3, col=2)
    fig.add_trace(go.Scatter(x=time, y=p_dot, mode='lines'), row=4, col=1)

    # Update layout
    fig.update_layout(title_text='Model Output', title_font_size=16, width=1000, height=1000)

    # Show the figure
    fig.show()

# Call the function with your model_out
plot_model_data(model_out)


In [6]:
# Define data and layouts for each plot
plots = [
    {'name': 'n',                        'color': 'blue',    'title': 'Plot of urban firm workforce n and n_target over time',  'row': 1, 'col': 1},
    {'name': 'F_target',                 'color': 'green',   'title': 'Plot of target number of firms over time',               'row': 2, 'col': 1},
    {'name': 'F',                        'color': 'red',     'title': 'Plot of number of firms and target over time',           'row': 2, 'col': 1},
    {'name': 'agglomeration_population', 'color': 'blue',    'title': 'Plot of urban workforce and population over time',       'row': 3, 'col': 1},
    {'name': 'N',                        'color': 'red',     'title': 'Plot of urban workforce and population over time',       'row': 3, 'col': 1},
    {'name': 'MPL',                      'color': 'black',   'title': 'Plot of urban wage and MPL over time',                   'row': 4, 'col': 1},
    {'name': 'wage',                     'color': 'red',     'title': 'Plot of urban wage and MPL over time',                   'row': 4, 'col': 1},
    {'name': 'subsistence_wage',         'color': 'black', 'dash': 'dot', 'title': 'Plot of urban wage and MPL over time',      'row': 4, 'col': 1},
    {'name': 'wage_premium',             'color': 'green',   'title': 'Plot of urban wage premium over time',                   'row': 5, 'col': 1},
    # {'name': 'y',                      'color': 'orange',  'title': 'Plot of firm output over time',                          'row': 5, 'col': 1},
    {'name': 'wage_premium',             'color': 'brown',   'title': 'Plot of urban wage premium over time',                   'row': 6, 'col': 1},
    {'name': 'k',                        'color': 'pink',    'title': 'Plot of urban firm capital over time',                   'row': 7, 'col': 1},
]

# Create subplots with increased spacing
fig = sp.make_subplots(
    rows=7, cols=1, shared_xaxes=True,
    subplot_titles=[
        plot['title'] for plot in plots
    ]
)

# Add traces for each plot with legends
for plot in plots:
    fig.add_trace(
        go.Scatter(x=time, y=model_out[plot['name']],
                   mode='lines', name=plot['name'],
                   line=dict(color=plot['color'], dash=plot.get('dash', 'solid'))),
        row=plot['row'], col=plot['col']
    )

    # Update axis titles
    fig.update_xaxes(title_text='Time Step', row=plot['row'], col=plot['col'])
    fig.update_yaxes(title_text=plot['name'], row=plot['row'], col=plot['col'])

# Update layout title
fig.update_layout(title_text='Model Output', height=1000)

# Show the plot
fig.show()
